# <div class="alert alert-info"> 1. Introduction </div>



# <div class="alert alert-info"> 2. Setup </div>

In [1]:
import joblib
import sys
import sklearn
import pandas as pd
import numpy as np

## for data
import collections
import json

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for processing
import spacy
nlp = spacy.load('en_core_web_sm')
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lst_stopwords = nltk.corpus.stopwords.words("english")

import string
import re
from bs4 import BeautifulSoup
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import  LabelEncoder

# for classification
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing

#from contractions import CONTRACTION_MAP
# import text_normalizer as tn

## for word embedding
import gensim
import gensim.downloader as gensim_api
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

## for sentiment analysis
from textblob import TextBlob

## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
# deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *



## for bert language model
import transformers

# utilities
#import nlp_utils as util

# Configuration
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

# display various ranges 
pd.options.display.float_format = '{:.3f}'.format

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))
print("Setup complete...")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Python: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
NLTK: 3.6.5
Scikit-learn: 0.23.2
Pandas: 1.3.3
Numpy: 1.21.2
Setup complete...


---
# <div class="alert alert-info"> 3. Data Preparation </div>

 ## <font color=red>**3.1.  News category dataset**

The dataset used is “News category dataset” from Kaggle (https://www.kaggle.com/rmisra/news-category-dataset. This dataset provided with around 200k news headlines from the year 2012 to 2018 obtained from HuffPost.

## <font color=red>3.2. Load data
    
The dataset is contained into a json file, so I will first read it into a list of dictionaries with json and then transform it into a pandas Dataframe.

In [2]:
df = pd.read_csv('../data/processed/News_Category.csv')
print(df.shape)
## print 5 random rows
df = df.reset_index(drop=True)
df.head(5)

(26768, 4)


,category,headline,short_description,text
0,BUSINESS,"U.S. Launches Auto Import Probe, China Vows To...",The investigation could lead to new U.S. tarif...,"U.S. Launches Auto Import Probe, China Vows To..."
1,BUSINESS,Starbucks Says Anyone Can Now Sit In Its Cafes...,The new policy was unveiled weeks after the co...,Starbucks Says Anyone Can Now Sit In Its Cafes...
2,BUSINESS,Seattle Passes Controversial New Tax On City's...,"Following the council vote, Amazon’s vice pres...",Seattle Passes Controversial New Tax On City's...
3,BUSINESS,Uber Ends Forced Arbitration In Individual Cas...,Victims will be free to go to court -- but a f...,Uber Ends Forced Arbitration In Individual Cas...
4,BUSINESS,"Chili's Hit By Data Breach, Credit And Debit C...",The breach is believed to have occurred betwee...,"Chili's Hit By Data Breach, Credit And Debit C..."


In [3]:
df.category.value_counts()

TRAVEL          9855
FOOD & DRINK    6208
BUSINESS        5878
SPORTS          4827
Name: category, dtype: int64

In [4]:
df = df[df['category'] == 'TRAVEL']

## <font color=red>**3.3. Text Processing and Normalizing** </font>  
    
Before feature engineering, we need to  pre-process, clean, and normalize text. Following are some of the popular pre-processing techniques:

**1. Text tokenization and lower casting**: Split doc into individual words and lower casting all words <br>
**2. Removing special characters**: remove special characters and punctuations <br>
**3. Removing stop words**: Words like "a" and "the" appear so frequently and  are called stop words, they can be filtered from the text to be processed <br>
**4. Stemming**: extract root of words by remove -ing, -ly, -ed... <br>
**5. Lemmatization**: In contrast to stemming, lemmatization looks beyond word reduction, and considers a language's full vocabulary to apply a *morphological analysis* to words. The lemma of 'was' is 'be' and the lemma of 'mice' is 'mouse'. Further, the lemma of 'meeting' might be 'meet' or 'meeting' depending on its use in a sentence. <br>

In [5]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=False, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   characters and then strip)
    #text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    #text = re.sub('[^a-zA-Z\s]', '', text)
    text = str(text).lower().strip()
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [6]:
lst_stopwords = nltk.corpus.stopwords.words("english")

df['text_clean'] = df["short_description"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=None))

df.head()

,category,headline,short_description,text,text_clean
5878,TRAVEL,"14 Ways To Make Family Road Trips Easier, From...",Having waterproof covers on the seats is kind ...,"14 Ways To Make Family Road Trips Easier, From...",having waterproof covers on the seats is kind ...
5879,TRAVEL,14 Trips To Take From New York City On A Long ...,"Charming towns, relaxing beaches and top hikin...",14 Trips To Take From New York City On A Long ...,"charming towns, relaxing beaches and top hikin..."
5880,TRAVEL,Disney Reveals Opening Seasons For 'Star Wars'...,Star Wars: Galaxy's Edge will open at Disneyla...,Disney Reveals Opening Seasons For 'Star Wars'...,star wars: galaxy's edge will open at disneyla...
5881,TRAVEL,Lonely Planet's Top European Destinations Of 2...,These underrated travel destinations in Europe...,Lonely Planet's Top European Destinations Of 2...,these underrated travel destinations in europe...
5882,TRAVEL,8 Majestic Islands In Europe That Most Tourist...,If you’re dreaming about a romantic European g...,8 Majestic Islands In Europe That Most Tourist...,if you’re dreaming about a romantic european g...


In [7]:
df.shape

(9855, 5)

---
# <div class="alert alert-info"> 4. SEMANTIC SEARCH/ DOC SIMILARITY </div>

## <font color=red>4.1. Find documents contains words

In [15]:
df.text_clean = df.text_clean.astype(str) 

sdf = df[df['text_clean'].str.contains("vietnam")]
print(sdf.shape)
sdf = sdf.reset_index()


for i in range(0,len(sdf)):
    print("\n", sdf.text_clean.iloc[i])

(16, 5)

 hoi an, vietnam vietnam’s capital hanoi is famous for its busy street life, french feel and colonial buildings and serves

 as a father, i am so excited that our two 20something children, andy and jackie, are embarking on a one-month adventure in indonesia, vietnam, and thailand.

 the question, "how did the tourist cross the road" is not a joke, it's a real query from most visitors to vietnam. to make it to the other side, you need to be brave, focused, and a risk-taker -- just like in life. here are 12 life lessons that you can learn from crossing the street in vietnam:

 i'm totally mesmerized by the traffic in hanoi, vietnam: one of the wonders of the world. it's a wonder not because, like so many of the world's cities, it's so terrible, but because it flows in the most extraordinary way. it's the triumph of a lack of system over a system.

 the following morning the vietnamese police showed up at his hotel room offering him a deal: he could be on the afternoon pan am fli

## <font color=red>4.2. Approach 1: Bag of Words + Hammer distance

### <font color=blue> BOW with count method

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,1), lowercase=False, binary='true')

In [28]:
# search_string1 = 'soho123, oxford, 34'
search_string = 'the variety of places to see and visit, the comfortable weather, the great street food and the sense of tourism all make northern vietnam an easy place to travel'
search_corpus = []
# standardize text data
search_string = re.sub(r'[^\w\s]', '', str(search_string).lower().strip()) 
# tokenize sentence to list of words
search_corpus = nltk.tokenize.word_tokenize(search_string)

In [27]:
df.head()

,category,headline,short_description,text,text_clean
5878,TRAVEL,"14 Ways To Make Family Road Trips Easier, From...",Having waterproof covers on the seats is kind ...,"14 Ways To Make Family Road Trips Easier, From...",having waterproof covers on the seats is kind ...
5879,TRAVEL,14 Trips To Take From New York City On A Long ...,"Charming towns, relaxing beaches and top hikin...",14 Trips To Take From New York City On A Long ...,"charming towns, relaxing beaches and top hikin..."
5880,TRAVEL,Disney Reveals Opening Seasons For 'Star Wars'...,Star Wars: Galaxy's Edge will open at Disneyla...,Disney Reveals Opening Seasons For 'Star Wars'...,star wars: galaxy's edge will open at disneyla...
5881,TRAVEL,Lonely Planet's Top European Destinations Of 2...,These underrated travel destinations in Europe...,Lonely Planet's Top European Destinations Of 2...,these underrated travel destinations in europe...
5882,TRAVEL,8 Majestic Islands In Europe That Most Tourist...,If you’re dreaming about a romantic European g...,8 Majestic Islands In Europe That Most Tourist...,if you’re dreaming about a romantic european g...


In [30]:
import scipy
from scipy.spatial import distance as pdist
from scipy.spatial import distance
#scipy.spatial.distance.cdist

# display various ranges 
pd.options.display.float_format = '{:.5f}'.format
vectorizer = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,1), lowercase=False, binary='true')

def calculate_similarity(data, search_corpus):
    dic_bow = vectorizer.fit_transform(data)
    bow_df = dic_bow.todense()
    bow_search_string = vectorizer.transform(search_corpus)
    bow_search_string = bow_search_string.todense()
    bow_search_string = bow_search_string * 2
    bow_search_string = bow_search_string - 1
    similarity_list = []
    for i in range(0, len(data)):
        similarity = 1  -  scipy.spatial.distance.cdist(np.array(bow_df[i]), np.array(bow_search_string), 'hamming')
        similarity_list.append(similarity.sum())  
        
    return similarity_list

result_df = df.copy()
result_df['score'] = calculate_similarity(df['text_clean'], search_corpus)

result_df.head()

,category,headline,short_description,text,text_clean,score
5878,TRAVEL,"14 Ways To Make Family Road Trips Easier, From...",Having waterproof covers on the seats is kind ...,"14 Ways To Make Family Road Trips Easier, From...",having waterproof covers on the seats is kind ...,0.00060
5879,TRAVEL,14 Trips To Take From New York City On A Long ...,"Charming towns, relaxing beaches and top hikin...",14 Trips To Take From New York City On A Long ...,"charming towns, relaxing beaches and top hikin...",0.00020
5880,TRAVEL,Disney Reveals Opening Seasons For 'Star Wars'...,Star Wars: Galaxy's Edge will open at Disneyla...,Disney Reveals Opening Seasons For 'Star Wars'...,star wars: galaxy's edge will open at disneyla...,0.00000
5881,TRAVEL,Lonely Planet's Top European Destinations Of 2...,These underrated travel destinations in Europe...,Lonely Planet's Top European Destinations Of 2...,these underrated travel destinations in europe...,0.00030
5882,TRAVEL,8 Majestic Islands In Europe That Most Tourist...,If you’re dreaming about a romantic European g...,8 Majestic Islands In Europe That Most Tourist...,if you’re dreaming about a romantic european g...,0.00000


In [31]:
result_df = result_df.sort_values(['score'], ascending=False).reset_index(drop=True)
for j in range(0, 15):
    print(j, result_df.text[j], result_df.score[j])

0 Six Days In Hanoi, Vietnam (PHOTOS). The variety of places to see and visit, the comfortable weather, the great street food and the sense of tourism all make northern Vietnam an easy place to travel. 0.0028999999999996806
1 Why Serendipity Is the Best Travel Guide. Patricia Schultz is the author of 1000 Places to See Before You Die, an opus that took her eight years to finish. And nothing will make you smarter about travel than covering the most sought-after destinations in the world. 0.0014999999999998348
2 5 Totally Amazing Europe Trips Under $500. The process of picking one place to visit in Europe, with its wealth of stunning travel destinations, isn't easy. But for budget travelers, we have a few excellent ideas for planning an unforgettable vacation that's both marvelous and within your means. 0.0014999999999998348
3 Ten Tips To Stay Safe While Traveling Abroad. I've been fortunate enough to travel all around the world but also witnessed an unbelievable array of street scams an

In [32]:
result_df.sort_values(['score'], ascending=False).head()

,category,headline,short_description,text,text_clean,score
0,TRAVEL,"Six Days In Hanoi, Vietnam (PHOTOS)","The variety of places to see and visit, the co...","Six Days In Hanoi, Vietnam (PHOTOS). The varie...","the variety of places to see and visit, the co...",0.00290
2,TRAVEL,5 Totally Amazing Europe Trips Under $500,The process of picking one place to visit in E...,5 Totally Amazing Europe Trips Under $500. The...,the process of picking one place to visit in e...,0.00150
3,TRAVEL,Ten Tips To Stay Safe While Traveling Abroad,I've been fortunate enough to travel all aroun...,Ten Tips To Stay Safe While Traveling Abroad. ...,i've been fortunate enough to travel all aroun...,0.00150
4,TRAVEL,Travel Bloggers Share Their 2013 Itineraries,"So many places to see, so little time. To help...",Travel Bloggers Share Their 2013 Itineraries. ...,"so many places to see, so little time. to help...",0.00150
1,TRAVEL,Why Serendipity Is the Best Travel Guide,Patricia Schultz is the author of 1000 Places ...,Why Serendipity Is the Best Travel Guide. Patr...,patricia schultz is the author of 1000 places ...,0.00150


## <font color=red>4.3. Approach 2: Word2Vec Embedding + Cosine similarity

In [33]:
# Set values for various parameters
feature_size = 300   # Word vector dimensionality
window_context= 30   # Context window size
min_word_count = 1  # Minimum word count
sample = 1e-3        # Downsample setting for frequent words

tokenized_corpus = []
for words in df['text_clean']:
    tokenized_corpus.append(words.split())
    
pretrained_model = Word2Vec(size=feature_size, window=window_context, min_count=min_word_count, workers=-1)
pretrained_model.build_vocab(tokenized_corpus)

In [34]:
pretrained_model.similarity('train','cat'),pretrained_model.similarity('train','travel'),  pretrained_model.similarity('dog','cat')

(0.019836832, 0.044482764, -0.04999336)

In [35]:
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
pretrained_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)
pretrained_model.train(tokenized_corpus, total_examples=pretrained_model.corpus_count, epochs=5)

(0, 0)

In [36]:
pretrained_model.similarity('train','cat'),pretrained_model.similarity('train','travel'),  pretrained_model.similarity('dog','cat')

(0.08540114, 0.29063696, 0.76094574)

In [37]:
def vectorize(corpus, model):
    # global embeddings
    embeddings = []
    for line in corpus:
        w2v = None
        count = 0
        for word in line.split():
            if word in model.wv.vocab:
                count += 1
                if w2v is None:
                    w2v = model.wv[word]
                else:
                    w2v = w2v + pretrained_model.wv[word]
        if w2v is not None:
            w2v = w2v / count
            # append element to the end of the embeddings list
            embeddings.append(w2v)
        
    return embeddings

In [39]:
embeddings = vectorize(df['text_clean'], pretrained_model)
X = pd.DataFrame(embeddings)
print(X.shape)
X.head()

(9855, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.00426,-0.03635,0.02960,0.04680,-0.06004,-0.04202,-0.01779,-0.07846,0.09440,0.06321,-0.00939,-0.04774,0.01888,-0.03199,-0.03435,0.05944,0.03639,0.08222,0.00347,-0.02145,-0.01539,0.05591,-0.09271,-0.00306,0.04559,-0.04124,-0.10490,0.14927,0.00612,-0.04978,-0.02111,0.08630,-0.02484,0.10281,0.09389,-0.08127,-0.04918,-0.01315,-0.00442,0.00928,0.04551,-0.01320,0.07764,0.01652,-0.08839,-0.05203,0.04161,-0.05629,0.08897,-0.07117,-0.01433,0.02634,0.04960,-0.04583,0.02045,0.05378,-0.03581,-0.04076,0.09460,-0.02135,-0.05479,0.02439,-0.03615,-0.18413,0.03412,0.00330,-0.07589,0.08939,0.01520,0.07521,0.12735,0.05883,0.11192,0.01041,-0.12005,-0.06200,0.09426,0.01862,0.02128,0.10111,0.02702,-0.10977,0.02525,0.06541,-0.05420,-0.11481,-0.00651,0.08521,-0.01602,-0.02513,-0.03600,-0.03554,-0.01480,-0.06265,-0.04482,-0.02752,0.00900,-0.02139,0.05039,0.03091,-0.00868,-0.00652,-0.03387,-0.02764,-0.12223,-0.02784,0.02550,-0.02619,-0.00402,-0.08895,0.01096,-0.01403,-0.03461,-0.01170,-0.01455,0.05003,0.02941,0.05630,0.02672,0.03564,-0.09514,-0.02293,-0.06559,0.08120,0.01197,-0.01608,-0.00287,-0.05050,0.09066,0.06155,-0.04609,-0.06041,-0.09583,0.01796,-0.03674,0.01179,0.03587,-0.03708,-0.02808,-0.01273,0.06286,0.02086,0.04499,-0.02956,0.02072,-0.00487,-0.08630,-0.07258,-0.02850,0.04011,0.06984,-0.00937,-0.02099,0.00585,0.05087,-0.02541,-0.10546,-0.05932,-0.05311,-0.01105,0.01674,-0.02521,0.05700,0.07857,-0.01716,-0.01357,0.00939,-0.00685,-0.03624,-0.03732,-0.09656,-0.08028,-0.02832,-0.05300,-0.04865,-0.03920,0.01852,-0.10464,-0.10242,0.02242,-0.08186,-0.03526,-0.01189,-0.03453,-0.07371,-0.03997,0.01175,-0.02640,0.13335,0.06887,0.02392,0.04668,-0.01511,0.00176,-0.06063,0.08916,-0.03800,-0.01514,-0.05440,-0.07298,0.04567,0.03231,-0.10043,0.02312,-0.03461,0.03033,-0.01201,-0.04632,0.04610,0.07273,0.00699,0.10201,-0.00457,0.03405,-0.03139,0.06292,0.13412,-0.05188,-0.08736,0.06148,-0.00068,-0.01788,-0.02777,0.07392,-0.03212,0.04732,0.03431,0.04084,0.00567,-0.03408,-0.03269,0.00124,0.08338,-0.00041,-0.00271,0.03468,-0.00251,-0.03419,0.05626,0.00467,0.09285,0.02413,0.04788,-0.07821,0.03726,-0.00489,-0.00854,-0.00360,-0.03645,-0.04861,-0.01989,-0.05999,0.01063,0.02437,-0.01000,-0.02233,0.01418,0.02701,-0.05853,0.00416,-0.03690,0.01580,0.01104,-0.01594,0.04733,0.14096,0.00262,-0.04789,-0.01906,-0.01830,-0.01005,0.07918,0.11737,0.04034,0.04208,-0.00471,0.02185,-0.09395,-0.05351,-0.06542,0.01636,-0.01562,0.03801,0.04777,0.05651,-0.03372,-0.01191,-0.01220,-0.01635,-0.00971,-0.11666,-0.01809,-0.04383,0.02182,-0.01684,0.03966,0.01486,0.01620,0.03419,0.03941
1,0.04208,0.04318,-0.02417,0.07605,0.02972,-0.03139,-0.09916,-0.06798,0.08361,0.06807,0.11671,-0.13859,-0.05581,0.02957,0.05223,0.04858,0.01049,0.01437,-0.00981,0.08252,0.03296,0.07274,-0.11195,0.02359,-0.02331,-0.03185,-0.03796,0.03839,0.05950,-0.03382,-0.01383,0.02360,0.02135,0.13750,-0.04786,-0.19461,-0.05352,-0.12066,0.01191,0.12074,-0.

In [40]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = range(0, 300))

X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.67068,0.23485,0.52427,0.41035,0.38287,0.48066,0.45462,0.43329,0.60989,0.40058,0.55693,0.57258,0.67326,0.57423,0.57203,0.57908,0.48222,0.63961,0.50355,0.65584,0.41126,0.65768,0.22623,0.47286,0.57540,0.42585,0.29556,0.85964,0.46279,0.39306,0.60997,0.50119,0.42455,0.64782,0.77535,0.48518,0.34747,0.33816,0.42117,0.27604,0.50045,0.64596,0.58793,0.68647,0.17841,0.48355,0.68380,0.23583,0.66772,0.19980,0.52604,0.41467,0.50943,0.35315,0.70761,0.71222,0.68128,0.62406,0.79871,0.58695,0.60380,0.36339,0.54340,0.08552,0.66837,0.61660,0.40802,0.70226,0.64869,0.62303,0.69017,0.58195,0.79391,0.34624,0.37883,0.40854,0.70314,0.52323,0.42519,0.72040,0.68802,0.34422,0.33686,0.80621,0.35408,0.20737,0.68006,0.40248,0.50383,0.42320,0.28960,0.41591,0.63286,0.49431,0.55730,0.46021,0.42979,0.44573,0.53592,0.72177,0.70404,0.56879,0.23476,0.50477,0.15491,0.44240,0.49719,0.43860,0.46064,0.39565,0.71843,0.51570,0.36627,0.46151,0.28567,0.62795,0.38978,0.75328,0.61850,0.56420,0.32919,0.45289,0.29156,0.61966,0.37568,0.57479,0.76371,0.37668,0.82287,0.66245,0.59155,0.42139,0.31098,0.39130,0.53042,0.33155,0.71084,0.37030,0.35392,0.32256,0.59303,0.68160,0.66843,0.50035,0.49800,0.39500,0.57188,0.44394,0.56007,0.77795,0.50892,0.55954,0.63253,0.40798,0.68917,0.65823,0.29082,0.38426,0.52800,0.66309,0.55713,0.37182,0.46619,0.50273,0.42991,0.70293,0.38387,0.52904,0.47420,0.48218,0.50929,0.15301,0.32381,0.52184,0.47154,0.29840,0.23183,0.31478,0.08754,0.68273,0.39156,0.54436,0.43572,0.26549,0.28424,0.33650,0.71964,0.20101,0.61605,0.60864,0.37622,0.53742,0.39943,0.48246,0.38800,0.50666,0.37560,0.43573,0.69390,0.39801,0.80274,0.51452,0.54805,0.63932,0.38068,0.55757,0.63050,0.41546,0.66522,0.69536,0.44251,0.84098,0.71651,0.50590,0.56939,0.79212,0.74282,0.25257,0.40322,0.56948,0.51836,0.33592,0.31749,0.83790,0.39949,0.63966,0.55995,0.67644,0.52452,0.36456,0.24172,0.48423,0.77774,0.45314,0.40934,0.54736,0.56634,0.49581,0.47020,0.50268,0.54830,0.60629,0.70141,0.34809,0.52979,0.42491,0.41786,0.37971,0.39510,0.41369,0.45975,0.22564,0.20756,0.41222,0.44004,0.52701,0.48274,0.68945,0.41204,0.72942,0.30890,0.59735,0.31058,0.73164,0.70977,0.81426,0.75476,0.38920,0.41096,0.61475,0.53074,0.61993,0.62988,0.42095,0.33797,0.63937,0.64415,0.42713,0.64904,0.31705,0.54732,0.51567,0.54933,0.43731,0.42258,0.36310,0.68696,0.38420,0.31034,0.22084,0.22218,0.34266,0.49004,0.59303,0.47702,0.61541,0.74828,0.62178,0.57069,0.57831
1,0.75548,0.56164,0.33400,0.51397,0.66865,0.51248,0.18690,0.46030,0.57727,0.41098,0.99456,0.25056,0.47919,0.72304,0.81803,0.54438,0.40702,0.43833,0.45787,0.96072,0.53512,0.70600,0.19373,0.56328,0.39344,0.45025,0.52077,0.52765,0.66742,0.44214,0.63255,0.36050,0.60832,0.73385,0.47930,0.19703,0.33357,0.14306,0.47216,0.59820,0.22799,0.32144,0.48492,0.74523,0.27545,0.27406,0.74957,0.35068,0.49993,0.51890,0.67689,0.53804,0.63966,0.60099,0.73817,0.40251,0.63607,0.56272,0.48009,0.52018,0.57979,0.35403,0.54198,0

In [42]:
all_item_simdf = pd.DataFrame(cosine_similarity(X),
                              index = X.index.tolist(),
                              columns = X.index.tolist())
all_item_simdf.head(3)

0       1       2       3       4       5       6       7       8       9       10      11      12      13      14      15      16      17      18      19      20      21      22      23      24      25      26      27      28      29      30      31      32      33      34      35      36      37      38      39      40      41      42      43      44      45      46      47      48      49      50      51      52      53      54      55      56      57      58      59      60      61    \
0 1.00000 0.92124 0.95529 0.94667 0.95412 0.95924 0.94510 0.96002 0.94569 0.94445 0.95572 0.94194 0.95632 0.93873 0.96945 0.96064 0.94690 0.96302 0.97099 0.95055 0.95646 0.96710 0.89518 0.96905 0.93413 0.93691 0.95783 0.96034 0.94850 0.94983 0.95909 0.94347 0.93853 0.95531 0.95311 0.95054 0.96170 0.93733 0.93899 0.94650 0.94784 0.96239 0.93798 0.86346 0.95245 0.95633 0.96535 0.96901 0.93631 0.96971 0.96853 0.96279 0.93779 0.95966 0.95186 0.95655 0.96641 0.94840 0.92417 0.91381 0.95910 0.95095   
1 0.92124 1.00000 0.92888 0.94355 0.94575 0.93478 0.91201 0.92947 0.92530 0.92050 0.92006 0.91422 0.93310 0.92044 0.93551 0.92278 0.91898 0.93623 0.93340 0.92897 0.93316 0.92897 0.91533 0.93429 0.89088 0.90736 0.93104 0.93655 0.90703 0.91070 0.92506 0.90885 0.90147 0.93208 0.92868 0.92877 0.92175 0.91402 0.92143 0.89682 0.91934 0.93779 0.90555 0.85304 0.93121 0.93296 0.93331 0.93235 0.90688 0.93091 0.93078 0.93387 0.91900 0.93326 0.92794 0.93312 0.93647 0.93004 0.92187 0.87797 0.93142 0.92486   
2 0.95529 0.92888 1.00000 0.95897 0.96736 0.96119 0.95536 0.95850 0.95820 0.94144 0.96032 0.94144 0.96692 0.94063 0.97283 0.96443 0.94426 0.96974 0.97130 0.95630 0.96696 0.96823 0.89457 0.97172 0.93641 0.93700 0.96695 0.96704 0.94525 0.95107 0.96364 0.93040 0.94163 0.96958 0.95497 0.96042 0.95320 0.95107 0.94824 0.93863 0.94991 0.96406 0.94803 0.89864 0.97740 0.96686 0.96926 0.97268 0.93992 0.96280 0.97255 0.96213 0.94794 0.96470 0.95189 0.96688 0.97367 0.95355 0.94804 0.90665 0.96341 0.95245   

     62      63      64      65      66      67      68      69      70      71      72      73      74      75      76      77      78      79      80      81      82      83      84      85      86      87      88      89      90      91      92      93      94      95      96      97      98      99      100     101     102     103     104     105     106     107     108     109     110     111     112     113     114     115     116     117     118     119     120     121     122     123   \
0 0.93859 0.96291 0.95157 0.96290 0.96438 0.96208 0.96529 0.94453 0.92833 0.94587 0.97000 0.95645 0.96408 0.95735 0.95993 0.95912 0.95719 0.93924 0.95095 0.96700 0.95465 0.96007 0.88814 0.96511 0.96406 0.94896 0.89465 0.92972 0.95940 0.96812 0.96228 0.95399 0.92536 0.95367 0.93429 0.96375 0.96561 0.93702 0.95918 0.95539 0.96625 0.95924 0.93323 0.95804 0.96249 0.96197 0.96560 0.95890 0.96266 0.96821 0.93091 0.92198 0.95173 0.94433 0.96278 0.96105 0.96072 0.96512 0.96314 0.92380 0.95590 0.95157   
1 0.93184 0.92767 0.91699 0.92464 0.93530 0.93698 0.93051 0.90934 0.90003 0.92032 0.94014 0.92683 0.93818 0.92743 0.94241 0.92383 0.94077 0.91305 0.92409 0.94325 0.92910 0.95877 0.86002 0.94045 0.94078 0.91451 0.88816 0.90410 0.93789 0.94628 0.92746 0.93313 0.89852 0.92593 0.90364 0.94086 0.93472 0.92083 0.93177 0.93461 0.94909 0.94174 0.92560 0.93578 0.92769 0.93575 0.94010 0.92469 0.92275 0.91940 0.89878 0.89945 0.91625 0.92746 0.93280 0.93433 0.93436 0.93438 0.94652 0.90940 0.94026 0.92350   
2 0.95790 0.96594 0.95819 0.96088 0.96067 0.96624 0.96853 0.93975 0.93673 0.95184 0.96984 0.95638 0.96440 0.96858 0.97331 0.96481 0.96838 0.94301 0.95269 0.97195 0.96100 0.96693 0.91424 0.97079 0.97013 0.95294 0.92702 0.92869 0.96845 0.97195 0.97239 0.96169 0.92625 0.96685 0.93130 0.97516 0.97198 0.95234 0.96290 0.95493 0.96387 0.97036 0.92895 0.96459 0.96776 0.96042 0.97121 0.96003 0.96123 0.95939 0.94451 0.91364 0.95307 0.95736 0.96821 0.96872 0.96871 0.96771 0.96693 0.94576 0.96200 0.95304